# **Mental Health Support ChatBOT GPT-NEO Finetuning**

## Installing packages

In [1]:
!pip install -q transformers datasets accelerate peft bitsandbytes wandb -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 86.1 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 23.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 84.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.2 MB/s 

## Loading Dataset

In [2]:
from datasets import Dataset 
import pandas as pd

In [3]:
dataset_path="/kaggle/input/emotion-dataset/emotion-emotion_69k.csv"

In [4]:
pandas_df=pd.read_csv(dataset_path,usecols=["Situation",'emotion','empathetic_dialogues','labels'])

In [5]:
df=Dataset.from_pandas(pandas_df)
df

Dataset({
    features: ['Situation', 'emotion', 'empathetic_dialogues', 'labels'],
    num_rows: 64636
})

## Preprocessing Data

In [6]:
from transformers import AutoTokenizer

In [7]:
model_name="EleutherAI/gpt-neo-2.7B"

In [8]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [9]:
# GPT NEO doesnot support pad token so we have to create one
tokenizer.pad_token=tokenizer.eos_token#"Use the same token for padding that is used for end-of-sequence."

In [10]:
def format_data(data):
    prompt=f"<|context|>{data['empathetic_dialogues']}<|endoftext|><|response|>{data['labels']}"
    
    return tokenizer(prompt,truncation=True,max_length=512)

In [11]:
tokenized_data=df.map(format_data,remove_columns=df.column_names,batched=False)

Map:   0%|          | 0/64636 [00:00<?, ? examples/s]

## Split Data

In [12]:
tokenized_data=tokenized_data.shuffle(seed=43)

In [13]:
split_data=tokenized_data.train_test_split(test_size=0.2)
train_data=split_data['train']
val_data=split_data['test']

In [14]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 51708
})

In [15]:
val_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 12928
})

## Prepare The Model

In [16]:
import torch
from transformers import AutoModelForCausalLM

In [17]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True,
    trust_remote_code=True
)

2025-07-30 07:31:02.230257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753860662.576896      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753860662.673656      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

In [23]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_API") 

wandb.login(key=my_secret)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maarijaqeel3200 (maarijaqeel3200-fast-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [26]:
from peft import LoraConfig, get_peft_model, TaskType

In [27]:
lora_config = LoraConfig(
    r=8,               # rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],  # GPT-Neo attention layers
    task_type=TaskType.CAUSAL_LM,
)

In [28]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  

trainable params: 3,276,800 || all params: 2,654,584,320 || trainable%: 0.1234


In [29]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/kaggle/working/empathy-gpt-neo-2.7b",
    overwrite_output_dir=True,

    per_device_train_batch_size=2,      # each T4
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,     # global batch ≈ 64
    num_train_epochs=3,

    learning_rate=2e-5,
    warmup_steps=200,
    lr_scheduler_type="cosine",

    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=50,
    save_total_limit=2,

    bf16=True,                          # T4 supports bfloat16
    dataloader_pin_memory=False,
    report_to="wandb",                   # or "wandb"
)

In [30]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    processing_class=tokenizer,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [31]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
500,2.033100,2.044685
1000,2.026600,2.019517


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:185: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during

KeyboardInterrupt: 

## Saving the Model

In [32]:
trainer.save_model("/kaggle/working/Mental_health_chatbot-gpt-neo")
tokenizer.save_pretrained("/kaggle/working/Mental_health_chatbot-gpt-neo")

('/kaggle/working/Mental_health_chatbot-gpt-neo/tokenizer_config.json',
 '/kaggle/working/Mental_health_chatbot-gpt-neo/special_tokens_map.json',
 '/kaggle/working/Mental_health_chatbot-gpt-neo/vocab.json',
 '/kaggle/working/Mental_health_chatbot-gpt-neo/merges.txt',
 '/kaggle/working/Mental_health_chatbot-gpt-neo/added_tokens.json',
 '/kaggle/working/Mental_health_chatbot-gpt-neo/tokenizer.json')

In [48]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()


## Quick Inference Test

In [49]:
from transformers import pipeline
chatbot = pipeline("text-generation",
                   model="/kaggle/working/Mental_health_chatbot-gpt-neo",
                   tokenizer="/kaggle/working/Mental_health_chatbot-gpt-neo",
                   device_map="auto")

prompt = "<|context|>Tommorrow is my exam. I am very depressed.<|endoftext|><|response|>"
print(chatbot(prompt,
              max_new_tokens=64,
              do_sample=True,
              temperature=0.7)[0]["generated_text"])

ValueError: Could not load model /kaggle/working/Mental_health_chatbot-gpt-neo with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.gpt_neo.modeling_gpt_neo.GPTNeoForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 292, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 600, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py", line 315, in _wrapper
   

## Push to HuggingFace

In [42]:
from huggingface_hub import notebook_login
notebook_login()          

In [50]:
!nvidia-smi


Wed Jul 30 12:28:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   48C    P0             27W /   70W |   14683MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [43]:
local_path="/kaggle/working/Mental_health_chatbot-gpt-neo"

In [44]:
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path=local_path,
    repo_id="Maarij-Aqeel/Mental_Health_chatbot",   # <— your Hub repo name
    repo_type="model",
)

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Maarij-Aqeel/Mental_Health_chatbot/commit/56e6160a518d9fee7125bab2a9a4d11e35b12634', commit_message='Upload folder using huggingface_hub', commit_description='', oid='56e6160a518d9fee7125bab2a9a4d11e35b12634', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Maarij-Aqeel/Mental_Health_chatbot', endpoint='https://huggingface.co', repo_type='model', repo_id='Maarij-Aqeel/Mental_Health_chatbot'), pr_revision=None, pr_num=None)

## Running the Model

In [ ]:
!pip install -U bitsandbytes 

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer
import torch

In [3]:
from peft import PeftModel, PeftConfig

2025-08-01 08:04:37.377948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754035477.566336      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754035477.621620      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
base=AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B",torch_dtype=torch.float16, device_map="auto")
model=PeftModel.from_pretrained(base,"Maarij-Aqeel/Mental_Health_chatbot")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/827 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['qalora_group_size', 'use_qalora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

In [5]:
tokenizer=AutoTokenizer.from_pretrained("Maarij-Aqeel/Mental_Health_chatbot")

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [6]:
prompt="<|context|>Tomorrow is my exam i am not feeling good.<|endoftext|><|response|>"
inputs=tokenizer(prompt,return_tensors="pt").to(model.device)

In [7]:
with torch.no_grad():
    out_ids = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
text=tokenizer.decode(out_ids[0], skip_special_tokens=True)
assistant=text.split("<|response|>")[1]
assistant

"I'm sorry to hear that. Maybe you should see a doctor.  You can't just hide from your problems!  It's hard to know what's wrong at times.  Maybe it's stress.  You can try meditation.  I wish you well on your exam today!  It will be hard but you'll be prepared for it.  \n"